# **RAG using Zephry-Beta model**

---

<a target="_blank" href="https://colab.research.google.com/github/samsonthomas951/Large-Language-Model-Notebooks/blob/main/2.%20Retrival%20Arguemented%20Generation%20with%20LLMs/Zephyr_RAG-llama_index.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>


intalling the required lybraries

In [1]:
!pip install llama-index transformers accelerate bitsandbytes 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.6/917.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninst

Load text and split into chunks:

## How to load your pdf to google colab.
1. Drag n Drop
 - select the pdf file that you want to ask questions about
 - drag it and drop it on the content folder on colab

2. Uploading via code
 - you can add a cell below and write these code
    ```py
        
        from google.colab import files
        #uploading a single file
        uploaded = files.upload()
    ```
 - using a document that is in your google drive
    ```py
        from google.colab import drive
        drive.mount('/content/drive')
    ```


In [2]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")
loader = PDFReader()
#path to your document 
documents = loader.load_data(file=Path('/content/Topic 6 Notes.pdf'))

In [3]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

downloading the emmbeding model

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [6]:
from llama_index import SummaryIndex

summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

In [7]:
from llama_index.response.notebook_utils import display_response

In [8]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [9]:
def query(question, response_mode):
    query_engine = vector_index.as_query_engine(response_mode=response_mode)
    response = query_engine.query(question)
    display_response(response)


Asking the questions

here are explanations about the response modes that are passed on the query function
* compact - fits much infromation as possible while staying below the model maximum input limitd

* refine - iteratively updates the answer to a query based on each document in the input set

* tree_summarize - this mode involves creating a tree summary from a set of selected candidate nodes(similar chunks)

In [10]:
query("what are the requirements of a data dictionary", "compact")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should se

**`Final Response:`** A data dictionary provides complete information about various aspects of a database system, including data flow, data structure, data elements, data processing, and data stores (both external and internal). Its requirements include:

1. Facilitating users and software designers by providing definitions of all data-related concepts.
2. Eliminating ambiguity and enabling synchronization of work for designers and programmers.
3. Serving as a basis for validating Data Flow Diagrams (DFDs).
4. Providing information about data flow, data structure, data elements, data processing, and data stores.
5. Including details such as primary and secondary names (aliases), use-cases, and control items for data elements.

Overall, a data dictionary is essential for ensuring accurate and consistent data usage in software development and implementation.

In [11]:
query("What does HIPO mean", "refine")

**`Final Response:`** HIPO (Hierarchical Input Process Output) is a method used to analyze and document software systems by depicting their modules in a hierarchical manner through diagrams. It involves classifying system functions into sub-functions and demonstrating them through HIPO diagrams, which provide a high-level view of the system's functions. While HIPO diagrams do not provide data flow or control flow information, they are useful for understanding the structure of the system in a graphical form. However, there is a possibility of different interpretations of diagrams and graphs, which can lead to misunderstandings. To overcome this, Structured English is used to provide detailed information about what is to be coded and how it is to be coded, reducing the gap of misunderstanding that may occur with diagrams or graphs. Structured English uses simple words of English in the concept of structured programming to provide precise information about what is to be coded and how it is to be coded.

In [12]:
query("How to create a dicision table", "tree_summarize")

**`Final Response:`** To create a decision table, follow these steps:
1. Identify all the conditions that are likely to be addressed.
2. Determine the relevant actions for each condition.
3. Create rules for each specific condition and action.
4. Define the actions for each rule.
5. Verify the decision table to remove any duplication of rules and actions.

Example:
Conditions/Actions  Rules
Conditions  Shows Connected  N N N N Y Y Y Y
Ping is Working  N N Y Y N N Y Y
Opens Website  Y N Y N Y N Y N
Actions  Check network cable  X

In this example, the decision table is for resolving the issue of internet connectivity. The conditions are listed under the column "Conditions," and the actions are listed under the column "Actions." The rules are listed under the "Rules" column, where "N" represents "No" and "Y" represents "Yes." The decision table helps in preventing errors by combining similar information and making decision-making more convenient.